In [1]:
!pip install rdkit > None #None для того, чтобы не видеть информация по загрузке

ERROR: Invalid requirement: '#None'


In [2]:
from rdkit import Chem

In [3]:
#Чтобы Future warning не портило визуализации
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
import pandas as pd
import numpy as np
import re
from rdkit.Chem.Descriptors import *
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [6]:
# нужные библиотеки
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

In [7]:
df = pd.read_csv(r'C:\Users\Ксения\Project\Megy\content\DATA_2.0.1.csv')


In [8]:
df

,link,Effective anisotropy constant (J/m3),magnetic anisotropy constant (J/m3),magnetocrystalline anisotropy constant (J/m3),Anisotropy constant,mass_magnetic_susceptibility(emu/Oe*g),"length, nm",Core_composition_formula,magnetization reversibility,magnetocrystalline anisotropy energy (MAE) (J/m3),...,Max/min(size),surface,pol,surf,Magnetic moment of core,LogP of organic coating,Number H acceptors of organic coating,Field amplitude,Field frequency,SAR
0,https://doi.org/10.1016/j.biomaterials.2014.04...,6000.0,34000.0,NaN,34000.0,NaN,14.0,Au1Fe2O3,NaN,NaN,...,1.0,NaN,NaN,NaN,2.262,0.000,0.0,126.00,0.32,74.000
1,https://doi.org/10.1039/d0na00820f,NaN,NaN,NaN,1500.0,NaN,7.1,Co0.03Fe0.97Fe2O4,NaN,NaN,...,0.0,NaN,NaN,NaN,2.387,-3.288,7.0,250.00,0.11,11.800
2,https://doi.org/10.1039/d0na00820f,NaN,NaN,NaN,2000.0,NaN,6.9,Co0.05Fe0.95Fe2O4,NaN,NaN,...,0.0,NaN,NaN,NaN,2.384,-3.288,7.0,250.00,0.11,13.300
3,http://dx.doi.org/10.1021/acs.chemmater.5b04780,NaN,NaN,NaN,82000.0,NaN,17.0,Co0.1Fe0.9Fe2O4,NaN,NaN,...,0.0,NaN,NaN,NaN,2.377,3.212,1.0,251.37,0.18,430.240
4,http://dx.doi.org/10.1021/acs.chemmater.5b04780,NaN,NaN,NaN,82000.0,NaN,17.0,Co0.1Fe0.9Fe2O4,NaN,NaN,...,0.0,NaN,NaN,NaN,2.377,3.212,1.0,201.12,0.30,213.743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,Работа ани,62000.0,223000.0,NaN,NaN,0.015,2.0,ZnFe2O4,NaN,2670000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
564,Работа ани,62000.0,223000.0,NaN,NaN,0.015,2.5,ZnFe2O4,NaN,2670000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
565,Работа ани,62000.0,223000.0,NaN,NaN,0.015,2.5,ZnFe2O4,NaN,2670000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566,Работа ани,62000.0,223000.0,NaN,NaN,0.015,5.5,ZnFe2O4,NaN,2670000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 41 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   link                                               568 non-null    object 
 1   Effective anisotropy constant (J/m3)               487 non-null    float64
 2   magnetic anisotropy constant (J/m3)                417 non-null    float64
 3   magnetocrystalline anisotropy constant (J/m3)      380 non-null    float64
 4   Anisotropy constant                                448 non-null    float64
 5   mass_magnetic_susceptibility(emu/Oe*g)             375 non-null    float64
 6   length, nm                                         568 non-null    float64
 7   Core_composition_formula                           568 non-null    object 
 8   magnetization reversibility                        322 non-null    float64
 9   magnetocry

Удалим лишние столбцы с сылками или с отсутсвующими данными

In [10]:
cols_to_drop = ['surface', 'pol', 'surf'] # TODO: 'link' - была в этом списке
df.drop(columns=cols_to_drop, inplace=True)

In [11]:
link = df['link']

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 38 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   link                                               568 non-null    object 
 1   Effective anisotropy constant (J/m3)               487 non-null    float64
 2   magnetic anisotropy constant (J/m3)                417 non-null    float64
 3   magnetocrystalline anisotropy constant (J/m3)      380 non-null    float64
 4   Anisotropy constant                                448 non-null    float64
 5   mass_magnetic_susceptibility(emu/Oe*g)             375 non-null    float64
 6   length, nm                                         568 non-null    float64
 7   Core_composition_formula                           568 non-null    object 
 8   magnetization reversibility                        322 non-null    float64
 9   magnetocry

Закодируем параметры типа object: 1.Core и Shell преобразуем в новые колонки с помощью rdkit 2.Crystalline сделаем с помощью 0 и 1, а вот  3. shape - наверное, стоит использовать LabelEncoder

# Анализ данных

In [13]:
df['Crystalline'] = [1 if i == 'yes' else 0 for i in df['Crystalline']]

Теперь формы: сферическая, кубическая и тому подобное закодированы цифрами

In [14]:
df['shape name'].unique()

array(['dimer', 'sphere', 'cube', nan, 'rod', 'cuboidal', 'toroid',
       'octahedral', 'cubic', 'polyhedral'], dtype=object)

188 - пропусков в MAE - это уже радует, но больше сократить количество пропусков вручную я не могу

In [15]:
df['magnetocrystalline anisotropy constant (J/m3)'].isnull().sum()

188

Эти данные либо маленькие, либо нулевые:

*   interdomain coupling (magnetoelectric coupling) (mV/cm Oe) - добавлены значения вручную в ноябре
*   pinning loss - добавлены значения в декабре
*   magnetization reversibility

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 38 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   link                                               568 non-null    object 
 1   Effective anisotropy constant (J/m3)               487 non-null    float64
 2   magnetic anisotropy constant (J/m3)                417 non-null    float64
 3   magnetocrystalline anisotropy constant (J/m3)      380 non-null    float64
 4   Anisotropy constant                                448 non-null    float64
 5   mass_magnetic_susceptibility(emu/Oe*g)             375 non-null    float64
 6   length, nm                                         568 non-null    float64
 7   Core_composition_formula                           568 non-null    object 
 8   magnetization reversibility                        322 non-null    float64
 9   magnetocry

Составим список с названием столбцов на всякий случай

In [17]:
features = ['Effective anisotropy constant (J/m3)', 'magnetic anisotropy constant (J/m3)', 'magnetocrystalline anisotropy constant (J/m3)',
            'Anisotropy constant ', 'Core_composition_formula', 'Crystalline', 'magnetocrystalline anisotropy energy (MAE) (J/m3)',
            'Curie temperature (K)', 'Spehricity', 'length, nm', 'exchange constant (J/m) *10^-12',
            'width, nm', 'depth, nm', 'shape name', 'Electronegativity core', 'Elongation', 'Flatness',
            'Space group', 'Demagnetization factors (Nd)', 'Saturation magnetization (emu/g)', 'Coercivity (Oe)',
            'Remanence magnetization (emu/g)', 'Shell composition (surface)',
            'interdomain coupling (alpha)',  'mass_magnetic_susceptibility(emu/Oe*g)']

Данные из колонки Max/min(size) не информативные, так теперь у нас есть колонка формы - правда колонка max/min более заполненная, но я не помню принцип её заполнения

Если бы я помнила можно было бы поступить наоборот, вроде там было что-то про *сферичность, если сфера 0, если нет 1 или наоборот*

In [18]:
df = df.loc[ :, features]
df

,Effective anisotropy constant (J/m3),magnetic anisotropy constant (J/m3),magnetocrystalline anisotropy constant (J/m3),Anisotropy constant,Core_composition_formula,Crystalline,magnetocrystalline anisotropy energy (MAE) (J/m3),Curie temperature (K),Spehricity,"length, nm",...,Elongation,Flatness,Space group,Demagnetization factors (Nd),Saturation magnetization (emu/g),Coercivity (Oe),Remanence magnetization (emu/g),Shell composition (surface),interdomain coupling (alpha),mass_magnetic_susceptibility(emu/Oe*g)
0,6000.0,34000.0,NaN,34000.0,Au1Fe2O3,0,NaN,NaN,NaN,14.0,...,1.0,1.0,167.0,0.333333,28.0,3.564,19.253,NaN,NaN,NaN
1,NaN,NaN,NaN,1500.0,Co0.03Fe0.97Fe2O4,0,NaN,NaN,1.0,7.1,...,1.0,1.0,227.0,0.333333,20.0,0.000,0.000,carboxymethylcellulose,NaN,NaN
2,NaN,NaN,NaN,2000.0,Co0.05Fe0.95Fe2O4,0,NaN,NaN,1.0,6.9,...,1.0,1.0,227.0,0.333333,20.0,0.000,0.000,carboxymethylcellulose,NaN,NaN
3,NaN,NaN,NaN,82000.0,Co0.1Fe0.9Fe2O4,1,NaN,NaN,1.0,17.0,...,1.0,1.0,227.0,0.333333,73.5,143.000,10.800,decanoic acid,NaN,NaN
4,NaN,NaN,NaN,82000.0,Co0.1Fe0.9Fe2O4,1,NaN,NaN,1.0,17.0,...,1.0,1.0,227.0,0.333333,86.2,9.100,69.000,decanoic acid,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,62000.0,223000.0,NaN,NaN,ZnFe2O4,0,2670000.0,600.0,NaN,2.0,...,1.0,1.0,227.0,0.333333,31.0,74.700,0.500,NaN,NaN,0.015
564,62000.0,223000.0,NaN,NaN,ZnFe2O4,0,2670000.0,600.0,NaN,2.5,...,1.0,1.0,227.0,0.333333,33.7,66.900,0.600,NaN,NaN,0.015
565,62000.0,223000.0,NaN,NaN,ZnFe2O4,0,2670000.0,600.0,NaN,2.5,...,1.0,1.0,227.0,0.333333,64.3,37.800,1.800,NaN,NaN,0.015
566,62000.0,223000.0,NaN,NaN,ZnFe2O4,0,2670000.0,600.0,NaN,5.5,...,1.0,1.0,227.0,0.333333,85.1,100.700,9.900,NaN,NaN,0.015


Надо исправить ошибку кодировки данных кристалличности. Получается, что все пропуски автоматически заполняются нулями


```
list_ = []

for i in df['Crystalline']:
    if i != '':
        if i == 'no':
            list_.append(0)
        else:
            list_.append(1)
    else:
        list_.append(None)
```
Не получилось, не понимаю, почему в список не получается добавить пустые значения



Посмотрим количнство пропусков в столбцах

In [19]:
#df['Crystalline'] = [1 if i == 'yes' else 0 if pd.notna(i) else np.nan for i in df['Crystalline']]

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 25 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Effective anisotropy constant (J/m3)               487 non-null    float64
 1   magnetic anisotropy constant (J/m3)                417 non-null    float64
 2   magnetocrystalline anisotropy constant (J/m3)      380 non-null    float64
 3   Anisotropy constant                                448 non-null    float64
 4   Core_composition_formula                           568 non-null    object 
 5   Crystalline                                        568 non-null    int64  
 6   magnetocrystalline anisotropy energy (MAE) (J/m3)  392 non-null    float64
 7   Curie temperature (K)                              485 non-null    float64
 8   Spehricity                                         458 non-null    float64
 9   length, nm

In [21]:
df = df.rename(columns = {'Shell composition (surface)': 'shell_comp'})

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 25 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Effective anisotropy constant (J/m3)               487 non-null    float64
 1   magnetic anisotropy constant (J/m3)                417 non-null    float64
 2   magnetocrystalline anisotropy constant (J/m3)      380 non-null    float64
 3   Anisotropy constant                                448 non-null    float64
 4   Core_composition_formula                           568 non-null    object 
 5   Crystalline                                        568 non-null    int64  
 6   magnetocrystalline anisotropy energy (MAE) (J/m3)  392 non-null    float64
 7   Curie temperature (K)                              485 non-null    float64
 8   Spehricity                                         458 non-null    float64
 9   length, nm

Чтобы получить данные из структуры ядра и кожуры, нужно добавить столбец с элементами из которых она состоит и стехиометрию. Сначала заполним эти столбцы нулями

In [23]:
df.insert(0, 'Elements', 0)
df.insert(0, 'Stoichiometry', 0)

In [24]:
for i, row in df.iterrows():
    df.Elements[i] = re.findall('[A-Z][a-z]?', df.Core_composition_formula[i])
    df.Stoichiometry[i] = re.findall('[\d.]+', df.Core_composition_formula[i])
    print(df.Elements[i])
    print(df.Stoichiometry[i])

for i, row in df.iterrows():
        r = df.Elements[i]
        for elem in range(len(r) - 1):
            if r[elem] == 'Fe' and r[elem + 1] == 'Fe':
                r[elem] = 'Fe2+' # Fe2+
            elif r[elem] == 'Fe':
                 r[elem] = 'Fe3+' # Fe3+
        print(r)
        df.Elements[i] = r

['Au', 'Fe', 'O']
['1', '2', '3']
['Co', 'Fe', 'Fe', 'O']
['0.03', '0.97', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.05', '0.95', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.1', '0.9', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.1', '0.9', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.1', '0.9', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.1', '0.9', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.1', '0.9', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.1', '0.9', '2', '4']
['Co', 'Mn', 'Fe', 'Fe', 'O']
['0.24', '0.15', '0.61', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.3', '0.7', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.3', '0.7', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.3', '0.7', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.3', '0.7', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.3', '0.7', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.5', '0.5', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.5', '0.5', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.5', '0.5', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.5', '0.5', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.5', '0.5', '2', '4']
['Co', 'Fe', 'F

C:\Users\Ксения\AppData\Local\Temp\ipykernel_28508\660171926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Elements[i] = re.findall('[A-Z][a-z]?', df.Core_composition_formula[i])
C:\Users\Ксения\AppData\Local\Temp\ipykernel_28508\660171926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Stoichiometry[i] = re.findall('[\d.]+', df.Core_composition_formula[i])
C:\Users\Ксения\AppData\Local\Temp\ipykernel_28508\660171926.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['1', '2', '4']
['Zn', 'Fe', 'O']
['2', '4']
['Zn', 'Fe', 'O']
['2', '4']
['Zn', 'Fe', 'O']
['2', '4']
['Zn', 'Fe', 'O']
['2', '4']
['Zn', 'Fe', 'O']
['2', '4']
['Zn', 'Fe', 'O']
['2', '4']
['Zn', 'Fe

In [25]:
for i, row in df.iterrows():
    df.Stoichiometry[i] = list(map(float, re.findall('[\d.]+', df.Core_composition_formula[i])))
    print(df.Stoichiometry[i])

[1.0, 2.0, 3.0]
[0.03, 0.97, 2.0, 4.0]
[0.05, 0.95, 2.0, 4.0]
[0.1, 0.9, 2.0, 4.0]
[0.1, 0.9, 2.0, 4.0]
[0.1, 0.9, 2.0, 4.0]
[0.1, 0.9, 2.0, 4.0]
[0.1, 0.9, 2.0, 4.0]
[0.1, 0.9, 2.0, 4.0]
[0.24, 0.15, 0.61, 2.0, 4.0]
[0.3, 0.7, 2.0, 4.0]
[0.3, 0.7, 2.0, 4.0]
[0.3, 0.7, 2.0, 4.0]
[0.3, 0.7, 2.0, 4.0]
[0.3, 0.7, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 

C:\Users\Ксения\AppData\Local\Temp\ipykernel_28508\3301359452.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Stoichiometry[i] = list(map(float, re.findall('[\d.]+', df.Core_composition_formula[i])))


[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.

Сделаем тоже самое для поверхности

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 27 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Stoichiometry                                      568 non-null    object 
 1   Elements                                           568 non-null    object 
 2   Effective anisotropy constant (J/m3)               487 non-null    float64
 3   magnetic anisotropy constant (J/m3)                417 non-null    float64
 4   magnetocrystalline anisotropy constant (J/m3)      380 non-null    float64
 5   Anisotropy constant                                448 non-null    float64
 6   Core_composition_formula                           568 non-null    object 
 7   Crystalline                                        568 non-null    int64  
 8   magnetocrystalline anisotropy energy (MAE) (J/m3)  392 non-null    float64
 9   Curie temp

Нужно ли делать это для покрытия, если количетсво пропусков для вещества покрытия больше 50%

Добавим колонку общего спина и посчитаем спин для каждой наночастицы отдельно

In [27]:
df.insert(0, 'common_spin', 0)

In [28]:
df = df.astype({'common_spin' : float})

Составим словарь спинов всех элементов, что у нас есть

In [29]:
df = pd.read_csv(r'C:\Users\Ксения\Project\Megy\add_spine.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         533 non-null    int64  
 1   id                 533 non-null    int64  
 2   orig_c1            533 non-null    object 
 3   orig_c2            322 non-null    object 
 4   c1                 533 non-null    object 
 5   c2                 322 non-null    object 
 6   c3                 9 non-null      object 
 7   formula_combined   533 non-null    object 
 8   chemical_formula   533 non-null    object 
 9   x                  533 non-null    float64
 10  y                  533 non-null    float64
 11  z                  509 non-null    float64
 12  sphericity         533 non-null    float64
 13  shape              533 non-null    int64  
 14  area/volume        533 non-null    float64
 15  max/min            533 non-null    int64  
 16  space_group_core   533 non

In [30]:
df['c2'].unique()

array(['Fe2 Co1 O4', nan, 'Ni1 O1', 'Fe1 Bi1 O3', 'Co1 O1', 'Fe3 O4',
       'Fe2 O3', 'Fe4 Bi2 O9', 'Fe1 O50', 'Fe1 O25', 'Co3 O4', 'Mn3 O4',
       'Fe2 Ni1 O4', 'Mn2 Co1 O4', 'Mn1 O1', 'Fe3 C1', 'Mg1 O1', 'Mo1',
       'Co1', 'Fe1'], dtype=object)

In [31]:
dic_spin = { 'Fe2+': 2, 'Fe3+': 5/2, 'Cu': 1/2, 'Zn': 0, 'Mn': 5/2, 'Co': 2,
             'Ni': 3/2, 'O': 0, 'Si': 0, 'Fe': 2, '': '',
             'Au': 1/2, 'C': 1/2, 'Mg': 0, 'Pd': 0 }

In [32]:
pd.options.mode.chained_assignment = None #чтобы убрать ошибку SettingWithCopyWarning

In [33]:
df.Elements

AttributeError: 'DataFrame' object has no attribute 'Elements'

In [ ]:
for i, _ in df.iterrows():
    sum_spin = 0

    # Проверьте, что Core_composition_formula[i] не равно 0, не пусто и имеет длину > 0
    if df.Core_composition_formula[i] != 0 and df.Core_composition_formula[i] != [] and len(df.Core_composition_formula[i]) != 0:

        # Убедитесь, что Elements[i] существует и является списком
        if not df.Elements[i] or not isinstance(df.Elements[i], list):
            continue

        # Перебираем элементы в Elements[i]
        for elem in df.Elements[i]:
            # Проверьте, что индекс элемента elem существует в Elements[i]
            if elem in df.Elements[i] and df.Elements[i].index(elem) < len(df.Stoichiometry[i]):
                ind = df.Elements[i].index(elem)
                sum_spin += df.Stoichiometry[i][ind] * dic_spin.get(elem, 0)  # Используем get() для безопасного доступа к dic_spin

        # Проверка на деление на ноль перед присвоением значения common_spin
        total_stoichiometry = sum(df.Stoichiometry[i])
        if total_stoichiometry != 0:
            df.at[i, 'common_spin'] = sum_spin / total_stoichiometry
        else:
            df.at[i, 'common_spin'] = 0 #nan


In [ ]:
df['common_spin'].unique()

array([0.91666667, 1.        , 1.01071429, 2.33333333, 0.78571429,
       0.33333333, 0.53571429, 1.17647059, 1.08333333, 1.05263158,
       1.07142857, 0.83333333, 1.38888889, 1.03571429, 0.96428571,
       0.92857143, 0.89285714, 1.66666667, 2.5       , 0.82142857,
       0.86170213, 0.75465839, 0.67127072, 0.60447761, 1.70807453,
       2.16666667, 0.        , 1.5       , 0.93714286, 0.85714286,
       0.91428571, 0.75      , 0.71428571])

In [ ]:
df.isna().sum() * 100 / len(df)

common_spin                                           0.000000
Stoichiometry                                         0.000000
Elements                                              0.000000
Effective anisotropy constant (J/m3)                 14.260563
magnetic anisotropy constant (J/m3)                  26.584507
magnetocrystalline anisotropy constant (J/m3)        33.098592
Anisotropy constant                                  21.126761
Core_composition_formula                              0.000000
Crystalline                                           0.000000
magnetocrystalline anisotropy energy (MAE) (J/m3)    30.985915
Curie temperature (K)                                14.612676
Spehricity                                           19.366197
length, nm                                            0.000000
exchange constant (J/m) *10^-12                      37.852113
width, nm                                             0.176056
depth, nm                                            12

In [ ]:
df.insert(0, 'org_coating_LogP', 0), df.insert(0, 'org_coating_HAcceptors', 0)
df = df.astype({'org_coating_LogP': float, 'org_coating_HAcceptors': float})

In [ ]:
LogP = {'Sodium citrate': 'C(C(=O)[O-])C(CC(=O)[O-])(C(=O)[O-])O',
        'Citric acid': 'OC(=O)CC(O)(C(=O)O)CC(=O)O',
        'DSPE': 'CCCCCCCCCCCCCCCCCC(O[C@@H](COP(OCCN)(O)=O)COC(CCCCCCCCCCCCCCCCC)=O)=O',
        'Alendronate': 'O=P(O)(O)C(O)(CCCN)P(=O)(O)O',
        'PMAO-PEG': 'CCCCCCCCCCCCCCCCC=C.C1=CC(=O)OC1=O',
        'PEG': 'OCCO', 'DHCA': 'OC(=O)CCc1ccc(O)c(O)c1',
        'HDA-G2': 'CCCCCCCCCCCCCCCCN', 'DMSA': 'O=C(O)[C@@H](S)[C@@H](S)C(=O)O',
        'PEG-350': 'OCCO', 'PEG-550': 'OCCO', 'PEG-1100': 'OCCO', 'PEG-2000': 'OCCO',
        'RGD': 'C1C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)N1)CCCN=C(N)N)CCCCN)CC2=CC=C(C=C2)O)CC(=O)O.C(=O)O.C(F)(F)F.C(=O)(C(F)(F)F)O',
        'poly(isobutylenemaleic) acid anhydride': 'CC(C)C(=O)OC(=O)C(C)C', 'Cysteamine': 'C(CS)N', 'PVP-10': 'O=C1N(C=C)CCC1',
        'PEG-phospholipid': 'CCCCCCCCCCCCCCCC(=O)OCC(COP(=O)([O-])OCC[N+](C)(C)C)OC(=O)CCCCCCCCCCCCCCC',
        'Oleic acid': 'CCCCCCCC\C=C/CCCCCCCC(O)=O', 'DSPE-PEG5k': 'CCCCCCCCC=CCCCCCCCC(=O)OCC(COP(=O)(O)OCCN)OC(=O)CCCCCCCC=CCCCCCCCC',
        'PCL5k-b-PEG5k': 'C1CCC(=O)OCC1', 'PEG-PMMA': 'CC(=C)C(=O)OC', 'DP-PEG': '[O-]P(=O)([O-])OP(=O)([O-])[O-]',
        'CC-PEG': 'Oc1c(O)cccc1', 'HX-PEG': 'CC(=O)NO',
        'DSPE-mPEG1000': 'CCCCCCCCCCCCCCCCCC(O[C@@H](COP(OCCN)(O)=O)COC(CCCCCCCCCCCCCCCCC)=O)=O',
        'DSPE-mPEG550': 'CCCCCCCCCCCCCCCCCC(O[C@@H](COP(OCCN)(O)=O)COC(CCCCCCCCCCCCCCCCC)=O)=O',
        'DSPE-mPEG750': 'CCCCCCCCCCCCCCCCCC(O[C@@H](COP(OCCN)(O)=O)COC(CCCCCCCCCCCCCCCCC)=O)=O',
        'DSPE-mPEG2000': 'CCCCCCCCCCCCCCCCCC(O[C@@H](COP(OCCN)(O)=O)COC(CCCCCCCCCCCCCCCCC)=O)=O',
        'DSPE-mPEG5000': 'CCCCCCCCCCCCCCCCCC(O[C@@H](COP(OCCN)(O)=O)COC(CCCCCCCCCCCCCCCCC)=O)=O',
        'Tween-80': 'CCCCCCCCC=CCCCCCCCC(=O)OCCOCC(C1C(C(CO1)OCCO)OCCO)OCCO', 'PEI': 'NCCN',
        'citrate': 'C(C(=O)[O-])C(CC(=O)[O-])(C(=O)[O-])O', 'GA(Gallic Acid)-PEG': 'C1=C(C=C(C(=C1O)O)O)C(=O)O',
        'poly- (maleic anhydride alt-1-octadecene)': 'CCCCCCCCCCCCCCCCC=C.C1=CC(=O)OC1=O',
        'citrate anions': 'C(C(=O)[O-])C(CC(=O)[O-])(C(=O)[O-])O',
        'decanoic acid': 'CCCCCCCCCC(=O)O', 'oleic acid- coated': 'CCCCCCCC\C=C/CCCCCCCC(O)=O',
        'carboxymethylcellulose': 'CC(=O)O.C(C(C(C(C(C=O)O)O)O)O)O',
        'Folic Acid': 'C1=CC(=CC=C1C(=O)NC(CCC(=O)O)C(=O)O)NCC2=CN=C3C(=N2)C(=O)N=C(N3)N',
        'Curcumin': 'COC1=C(C=CC(=C1)C=CC(=O)CC(=O)C=CC2=CC(=C(C=C2)O)OC)O',
        'phosphatidylcholine': 'CCCCCCCCCCCCCCCCCCCCCCCC(=O)OC(COC(=O)CCCCCCCC=CCCCCCC)COP(=O)([O-])OCC[N+](C)(C)C',
		'oleic acid': 'CCCCCCCC\C=C/CCCCCCCC(O)=O', 'DSPE-PEG5k': 'CCCCCCCCC=CCCCCCCCC(=O)OCC(COP(=O)(O)OCCN)OC(=O)CCCCCCCC=CCCCCCCCC',
		'chitosan': 'COC(=O)NC1C(C(C(OC1OC2C(OC(C(C2O)N)OC3C(OC(C(C3O)N)O)CO)CO)CO)OC4C(C(C(C(O4)CO)OC5C(C(C(C(O5)CO)OC6C(C(C(C(O6)CO)OC7C(C(C(C(O7)CO)OC8C(C(C(C(O8)CO)OC9C(C(C(C(O9)CO)O)O)N)O)N)O)N)O)N)O)N)O)N)O',
		'citric acid': 'OC(=O)CC(O)(C(=O)O)CC(=O)O', 'folate chitosan': 'C1=CC(=CC=C1C(=O)NC(CCC(=O)O)C(=O)O)NCC2=CN=C3C(=N2)C(=O)NC(=N3)N',
        'N-(trimethoxysilylpropyl)-ethylenediaminetriacetate': 'C(CN(CCN(CC(=O)O)CC(=O)O)CC(=O)O)C[Si](O)(O)O', 'triethylene glycol': 'C(COCCOCCO)O',
		'meso-2,3-dimercaptosuccinic acid': 'O=C(O)[C@@H](S)[C@@H](S)C(=O)O',
		'TMAH':'C[N+](C)(C)C.[OH-]', 'poly(2-hydroxyethyl aspartamide)':'C(CO)NC(=O)C(CC(=O)N)N', 'citric acid': 'C(C(=O)O)C(CC(=O)O)(C(=O)O)O',
		'Pluronic F-127': 'CC(COCCO)OCCO', 'DTPMP':'C(CN(CP(=O)(O)O)CP(=O)(O)O)N(CCN(CP(=O)(O)O)CP(=O)(O)O)CP(=O)(O)O', 'Chitosan':'COC(=O)NC1C(C(C(OC1OC2C(OC(C(C2O)N)OC3C(OC(C(C3O)N)O)CO)CO)CO)OC4C(C(C(C(O4)CO)OC5C(C(C(C(O5)CO)OC6C(C(C(C(O6)CO)OC7C(C(C(C(O7)CO)OC8C(C(C(C(O8)CO)OC9C(C(C(C(O9)CO)O)O)N)O)N)O)N)O)N)O)N)O)N)O',
		'Gly': 'C(C(=O)O)N', 'Lys':'C(CCN)CC(C(=O)O)N', 'Trp': 'C1=CC=C2C(=C1)C(=CN2)CC(C(=O)O)N', 'polyaspartic acid': 'C(C(C(=O)O)N)C(=O)O',
		'PVA': 'C=COC(C)=O'}

In [ ]:
for i, _ in df.iterrows(): #перебираем строки таблицы
    if df.shell_comp[i] in LogP.keys(): #условие на то, что наше покрытие органическое
        m = Chem.MolFromSmiles(LogP[df.shell_comp[i]]) #вытаскиваем молекулу из SMILES представления, используя модуль rdkit
        df.org_coating_HAcceptors[i] = NumHAcceptors(m) #считаем для неё число акцепторов водорода
        df.org_coating_LogP[i] = MolLogP(m) #и липофильность

In [ ]:
df.org_coating_HAcceptors.unique()

array([0., 7., 1., 9., 8., 4., 3.])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 30 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   org_coating_HAcceptors                             568 non-null    float64
 1   org_coating_LogP                                   568 non-null    float64
 2   common_spin                                        568 non-null    float64
 3   Stoichiometry                                      568 non-null    object 
 4   Elements                                           568 non-null    object 
 5   Effective anisotropy constant (J/m3)               487 non-null    float64
 6   magnetic anisotropy constant (J/m3)                417 non-null    float64
 7   magnetocrystalline anisotropy constant (J/m3)      380 non-null    float64
 8   Anisotropy constant                                448 non-null    float64
 9   Core_compo

Пересмотрим описание морфологии частицы, чтобы избежать сильного влияния этих параемтров на друг друга

In [ ]:
df.insert(0, 'volume', 0)
df.insert(0, 'area', 0)
df = df.astype({'volume': float, 'area': float})

In [ ]:
df['shape name'].unique()

array(['dimer', 'sphere', 'cube', nan, 'rod', 'cuboidal', 'toroid',
       'octahedral', 'cubic', 'polyhedral'], dtype=object)

In [ ]:
df = df.rename(columns = {'shape name': 'shape_name'})


Можно было бы использовать для вычисления объема и площади поверхности



In [ ]:
for i, row in df.iterrows(): #перебираем построчно
  #если мы имеем околосферические частицы или материал очень странной формы (Nanoflower), или значение формы пропущено,
  #то используем формулу подсчета объема сферы:
    if df.shape_name[i] in ['spherical', 'sphere', 'Nanoflower', 'Spherical', 'sphere', 'Micelle',
                            'Sphere on sphere', 'sperical', 'Sandwich-like', 'Yolk-shell', 'spherical'] or str(df.shape_name[i]) == 'nan': #условие
       df.volume[i] = 4.19 * float(row['length, nm']) ** 3 #подсчет объема
       df.area[i] = 12.57 * float(row['length, nm']) ** 2 #площадь
      #если мы имеем частицы с формой параллелепипеда то используем соответствующую формулу для подсчета:
    elif df.shape_name[i] in ['Plate', 'cuboidal', 'dimer', "Cubic on the cubic's corners", 'Cubic',
                              'Nanowhiskers', 'cubic', 'cube', 'Rod', 'rod', 'Rhombohedron']: #формула
       df.volume[i] = float(row['length, nm']) * float(row['length, nm']) * float(row['width, nm']) #непосредственно подсчет объема
       df.area[i] = 2 * float(row['length, nm']) * float(row['depth, nm']) + 4 * float(row['length, nm']) * float(row['width, nm'])
    elif df.shape_name[i] in ['Hexaplate', 'hexagon']: #аналогично для гексагональных частиц
       df.volume[i] = 2.60 * float(row['length, nm'])**2 * float(row['width, nm'])
       df.area[i] = 2 * 2.60 * float(row['length, nm'])**2 + 6 * float(row['length, nm']) * float(row['width, nm'])
    elif df.shape_name[i] in ['Octapod', 'octopod', 'Octapode', 'octahedral']:
       df.volume[i] = 0.47 * float(row['length, nm'])**3
       df.area[i] = 3.46 * float(row['length, nm'])**2
    elif df.shape_name[i] in ['Tetrahedron', 'tetrahedron', 'polyhedral']: #и для тетраэдров
       df.volume[i] = 0.118 * float(row['length, nm'])**3
       df.area[i] = 1.73 * float(row['length, nm'])**2

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 32 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   area                                               498 non-null    float64
 1   volume                                             567 non-null    float64
 2   org_coating_HAcceptors                             568 non-null    float64
 3   org_coating_LogP                                   568 non-null    float64
 4   common_spin                                        568 non-null    float64
 5   Stoichiometry                                      568 non-null    object 
 6   Elements                                           568 non-null    object 
 7   Effective anisotropy constant (J/m3)               487 non-null    float64
 8   magnetic anisotropy constant (J/m3)                417 non-null    float64
 9   magnetocry

In [ ]:
df['volume']

0      2744.00000
1      1499.64709
2      1376.45271
3      4913.00000
4      4913.00000
          ...    
563      33.52000
564      65.46875
565      65.46875
566     697.11125
567           NaN
Name: volume, Length: 568, dtype: float64

In [ ]:
df['area']

0      1176.0000
1       633.6537
2       598.4577
3      1734.0000
4      1734.0000
         ...    
563      50.2800
564      78.5625
565      78.5625
566     380.2425
567          NaN
Name: area, Length: 568, dtype: float64

# Посмотрим на распредение данных и на количество пропусков

In [ ]:
df.isna().sum() * 100 / len(df)

area                                                 12.323944
volume                                                0.176056
org_coating_HAcceptors                                0.000000
org_coating_LogP                                      0.000000
common_spin                                           0.000000
Stoichiometry                                         0.000000
Elements                                              0.000000
Effective anisotropy constant (J/m3)                 14.260563
magnetic anisotropy constant (J/m3)                  26.584507
magnetocrystalline anisotropy constant (J/m3)        33.098592
Anisotropy constant                                  21.126761
Core_composition_formula                              0.000000
Crystalline                                           0.000000
magnetocrystalline anisotropy energy (MAE) (J/m3)    30.985915
Curie temperature (K)                                14.612676
Spehricity                                           19

In [ ]:
core_formula = df['Core_composition_formula']

Можно заполнить пропуски в колонках, где пропусков не более 15% от общего числа данных

Нужно не забыть убрать повторение столбца interdomain coupling - в документе с данными, есть только один такой столбец, но в colab их два. Оказалось, что это не ошибка исходного документа


# Заполнение пропусков KNN

In [ ]:
nan_before_knn = df.isna().sum().sum()
print(f"Number of NaN values before KNN imputation: {nan_before_knn}")

Number of NaN values before KNN imputation: 2145


Список колонк, в которых есть пропуски

In [ ]:
colomn_turn = ['area', 'volume', 'org_coating_HAcceptors',
               'Effective anisotropy constant (J/m3)', 'magnetic anisotropy constant (J/m3)',
               'magnetocrystalline anisotropy constant (J/m3)', 'Anisotropy constant',
               'Curie temperature (K)', 'exchange constant (J/m) *10^-12', 'magnetocrystalline anisotropy energy (MAE) (J/m3)',
               'Electronegativity core', 'Space group', 'Demagnetization factors (Nd)',
               'Saturation magnetization (emu/g)', 'Coercivity (Oe)',
               'Remanence magnetization (emu/g)', 'interdomain coupling (alpha)',
               'mass_magnetic_susceptibility(emu/Oe*g)']

Название колонки, которую можно почистить и можно поменять, а также процент данных, которые нужно удалить можно уменьшить или увлечить в по желанию

In [ ]:
df_copy = df.copy()

In [ ]:
# Заполним значения с помощью KNN
# Удалим ненужные столбцы из датасета для применения kNN
df_for_kNN = df_copy.drop(columns=['Core_composition_formula', 'shell_comp', 'shape_name', 'Stoichiometry', 'Elements'])  # Уберите любые другие столбцы, которые вам не нужны для анализа

# Нормализация данных
scaler = StandardScaler()
normalized_data = scaler.fit_transform(df_for_kNN)

# Заполним пропущенные значения в датасете с помощью kNN
imputer = KNNImputer(n_neighbors=4)
imputed_data = imputer.fit_transform(normalized_data)
df_imputed = pd.DataFrame(imputed_data, columns=df_for_kNN.columns)

# Объединим заполненные данные с исходным датасетом
df_filled = pd.concat([df_copy[['Core_composition_formula']], df_imputed], axis=1)

# Выведем информацию о пропущенных значениях после заполнения
print(f'Процент пропущенных значений после заполнения: \n{df_filled.isna().sum() * 100 / len(df_filled)}')

Процент пропущенных значений после заполнения: 
Core_composition_formula                             0.0
area                                                 0.0
volume                                               0.0
org_coating_HAcceptors                               0.0
org_coating_LogP                                     0.0
common_spin                                          0.0
Effective anisotropy constant (J/m3)                 0.0
magnetic anisotropy constant (J/m3)                  0.0
magnetocrystalline anisotropy constant (J/m3)        0.0
Anisotropy constant                                  0.0
Crystalline                                          0.0
magnetocrystalline anisotropy energy (MAE) (J/m3)    0.0
Curie temperature (K)                                0.0
Spehricity                                           0.0
length, nm                                           0.0
exchange constant (J/m) *10^-12                      0.0
width, nm                               

In [ ]:
print(df_filled)

    Core_composition_formula      area    volume  org_coating_HAcceptors  \
0                   Au1Fe2O3 -0.075955 -0.052851               -0.565192   
1          Co0.03Fe0.97Fe2O4 -0.075992 -0.052852                2.957069   
2          Co0.05Fe0.95Fe2O4 -0.075994 -0.052852                2.957069   
3            Co0.1Fe0.9Fe2O4 -0.075917 -0.052851               -0.062012   
4            Co0.1Fe0.9Fe2O4 -0.075917 -0.052851               -0.062012   
..                       ...       ...       ...                     ...   
563                  ZnFe2O4 -0.076031 -0.052852               -0.565192   
564                  ZnFe2O4 -0.076029 -0.052852               -0.565192   
565                  ZnFe2O4 -0.076029 -0.052852               -0.565192   
566                  ZnFe2O4 -0.076009 -0.052852               -0.565192   
567                  ZnFe2O4 -0.076020 -0.052852               -0.565192   

     org_coating_LogP  common_spin  Effective anisotropy constant (J/m3)  \
0          

Как видно Nan значения в первых двух колонках, которыее далее не используются

In [ ]:
nan_after_knn = new.isna().sum()
print(f"Number of NaN values after KNN imputation: {nan_after_knn}")

Number of NaN values after KNN imputation: Core_composition_formula                               0
shell_comp                                           325
shape_name                                            90
Stoichiometry                                          0
Elements                                               0
area                                                   0
volume                                                 0
org_coating_HAcceptors                                 0
org_coating_LogP                                       0
common_spin                                            0
Effective anisotropy constant (J/m3)                   0
magnetic anisotropy constant (J/m3)                    0
magnetocrystalline anisotropy constant (J/m3)          0
Anisotropy constant                                    0
Crystalline                                            0
magnetocrystalline anisotropy energy (MAE) (J/m3)      0
Curie temperature (K)                        

Как видим пропусков теперь нет, но элементов осталось 529 из 568

In [ ]:
df['link'] = link #TODO: эту тоже

In [ ]:
df_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 28 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Core_composition_formula                           568 non-null    object 
 1   area                                               568 non-null    float64
 2   volume                                             568 non-null    float64
 3   org_coating_HAcceptors                             568 non-null    float64
 4   org_coating_LogP                                   568 non-null    float64
 5   common_spin                                        568 non-null    float64
 6   Effective anisotropy constant (J/m3)               568 non-null    float64
 7   magnetic anisotropy constant (J/m3)                568 non-null    float64
 8   magnetocrystalline anisotropy constant (J/m3)      568 non-null    float64
 9   Anisotropy

In [ ]:
# Преобразование обратно в наши значения
org_copy = scaler.inverse_transform(df_filled.iloc[:, 1:]) # Без столбца id
df_recopy = pd.DataFrame(org_copy, columns=df_filled.columns[1:]) # Данные в исходном виде

df_recopy['id'] = id
print(df_recopy['Coercivity (Oe)'].unique())

[3.56400e+00 0.00000e+00 1.43000e+02 9.10000e+00 7.40000e+00 9.41200e+01
 1.05970e+01 1.35800e+01 2.54500e+00 1.57010e+01 7.36000e+00 4.00000e+02
 3.00000e+02 1.08600e+01 1.10000e+01 5.00000e+00 8.00000e+00 6.43000e+01
 5.56000e+01 2.00000e+01 8.00000e-01 1.23800e+02 1.00000e+01 1.45000e+03
 6.31500e+02 7.48720e+02 1.56715e+03 7.97000e+02 2.77400e+02 1.27200e+02
 4.70000e+01 7.73000e+01 5.91000e+01 1.95400e+02 7.25000e+01 1.01775e+02
 7.53000e+01 4.00000e+00 1.68300e+01 6.11000e+00 6.00000e+01 2.24000e+01
 6.80000e+01 5.73000e+01 1.30000e+02 7.20000e+00 7.50000e+00 1.20000e+02
 2.00000e+02 2.03000e+02 2.60000e+01 2.30000e+01 2.80000e+01 7.00000e+00
 4.04667e+02 1.84800e+00 1.73667e+02 2.20000e+01 3.70000e+01 8.00000e+01
 1.18000e+02 6.12000e+02 1.01720e+03 1.46500e+03 1.92280e+03 3.56600e+00
 7.22400e+00 1.00000e+02 1.50000e+04 2.16000e+04 3.08000e+04 7.50000e+02
 8.80000e-01 2.50000e-01 4.45200e+01 1.00500e+03 5.38000e+01 1.28500e+02
 1.14400e+02 7.58000e+01 2.52200e+02 1.52400e+02 5.

In [ ]:
df = df_recopy.copy()

Сохраним копию изменений в формате csv

In [ ]:
df

,area,volume,org_coating_HAcceptors,org_coating_LogP,common_spin,Effective anisotropy constant (J/m3),magnetic anisotropy constant (J/m3),magnetocrystalline anisotropy constant (J/m3),Anisotropy constant,Crystalline,...,Elongation,Flatness,Space group,Demagnetization factors (Nd),Saturation magnetization (emu/g),Coercivity (Oe),Remanence magnetization (emu/g),interdomain coupling (alpha),mass_magnetic_susceptibility(emu/Oe*g),id
0,1176.000000,2744.000000,0.0,0.0000,0.916667,6000.0,34000.0,11000.0,34000.00,0.0,...,1.0,1.0,167.0,0.333333,28.0,3.564,19.253,0.500,4.712388e-10,<built-in function id>
1,633.653700,1499.647090,7.0,-3.2879,1.000000,11000.0,128.0,47750.0,1500.00,0.0,...,1.0,1.0,227.0,0.333333,20.0,0.000,0.000,0.500,2.144137e-07,<built-in function id>
2,598.457700,1376.452710,7.0,-3.2879,1.000000,11000.0,128.0,47750.0,2000.00,0.0,...,1.0,1.0,227.0,0.333333,20.0,0.000,0.000,0.500,2.144137e-07,<built-in function id>
3,1734.000000,4913.000000,1.0,3.2117,1.000000,9625.0,167276.0,11000.0,82000.00,1.0,...,1.0,1.0,227.0,0.333333,73.5,143.000,10.800,0.500,3.750000e-03,<built-in function id>
4,1734.000000,4913.000000,1.0,3.2117,1.000000,230100.0,104.0,1613500.0,82000.00,1.0,...,1.0,1.0,227.0,0.333333,86.2,9.100,69.000,4.714,1.336785e-04,<built-in function id>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,50.280000,33.520000,0.0,0.0000,1.666667,62000.0,223000.0,166000.0,133196.56,0.0,...,1.0,1.0,227.0,0.333333,31.0,74.700,0.500,9.100,1.500000e-02,<built-in function id>
564,78.562500,65.468750,0.0,0.0000,1.666667,62000.0,223000.0,166000.0,133196.56,0.0,...,1.0,1.0,227.0,0.333333,33.7,66.900,0.600,9.100,1.500000e-02,<built-in function id>
565,78.562500,65.468750,0.0,0.0000,1.666667,62000.0,223000.0,158000.0,133196.56,0.0,...,1.0,1.0,227.0,0.333333,64.3,37.800,1.800,9.100,1.500000e-02,<built-in function id>
566,380.242500,697.111250,0.0,0.0000,1.666667,62000.0,223000.0,122000.0,133196.56,0.0,...,1.0,1.0,227.0,0.333333,85.1,100.700,9.900,9.100,1.500000e-02,<built-in function id>


# Корреляция

In [35]:
sns.set (font_scale= 2)
corr_matrix = df.corr()
mask = np.zeros_like(corr_matrix, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(50, 55))
heatmap = sns.heatmap(corr_matrix, mask=mask, square=True, linewidths=0.5,
					  cmap=sns.color_palette("coolwarm", as_cmap=True), cbar_kws = {'shrink': 0.6, 'ticks': [-1, -0.5, 0, 0.5, 1]}, #для изменения цвета
												   vmin = -1, vmax = 1, annot = True, annot_kws ={'size': 20}) #annot=False удалит данные из DATA
plt.show()

AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

Посмотрим корреляцию данных, как можно заметить больше всего накладываются друг на друга данные связанные с геометрическими параметрами частиц, так как у нас уже есть объём и площадь частицы, может можно отбросиь эти данные

Создадим на их основе параметры длина/ширину или плозадь на объём

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 28 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   area                                               568 non-null    float64
 1   volume                                             568 non-null    float64
 2   org_coating_HAcceptors                             568 non-null    float64
 3   org_coating_LogP                                   568 non-null    float64
 4   common_spin                                        568 non-null    float64
 5   Effective anisotropy constant (J/m3)               568 non-null    float64
 6   magnetic anisotropy constant (J/m3)                568 non-null    float64
 7   magnetocrystalline anisotropy constant (J/m3)      568 non-null    float64
 8   Anisotropy constant                                568 non-null    float64
 9   Crystallin

In [ ]:
df.insert(0, 'Max/min(size)', 0)
df.insert(0, 'Area/volume', 0)

In [ ]:
df = df.astype({'Max/min(size)': float, 'Area/volume': float})

In [ ]:
df = df.rename(columns = {'length, nm': 'length', 'width, nm': 'width', 'depth, nm': 'depth'})

In [ ]:
for i, _ in df.iterrows():
    maxim = np.max(np.array([df.length[i], df.width[i], df.depth[i]])) #берем максимальное значение из трех
    minim = np.min(np.array([df.length[i], df.width[i], df.depth[i]])) #берем минимальное значение из трех
    df['Max/min(size)'][i] = maxim/minim
    df['Area/volume'][i] = df.area[i]/df.volume[i] #находим отношение объема к площади

C:\Users\Ксения\AppData\Local\Temp\ipykernel_13712\2741303420.py:5: RuntimeWarning: invalid value encountered in scalar divide
  df['Area/volume'][i] = df.area[i]/df.volume[i] #находим отношение объема к площади


In [ ]:
df['Area/volume'].unique()

array([4.28571429e-01, 4.22535211e-01, 4.34782609e-01, 3.52941176e-01,
       5.00000000e-01, 1.57894737e-01, 3.15789474e-01, 3.33333333e-01,
       3.00000000e-01, 2.50000000e-01, 2.22222222e-01, 2.72727273e-01,
       2.72727273e-01, 3.89610390e-01, 2.50836120e-01, 7.50000000e-01,
       4.00000000e-01, 6.52173913e-01, 1.46341463e+00, 4.34782609e-01,
       2.44897959e-01, 1.86915888e-01, 1.32743363e-01, 1.82038835e-02,
       8.57142857e-01, 2.00000001e+00, 1.50000000e+00, 1.20000000e+00,
       6.00000000e-01, 2.92682927e-01, 5.21739130e-01, 1.50753769e-01,
       6.25000000e-03, 4.28571429e-01, 3.75000000e-01, 8.57142857e-02,
       1.76470588e-01, 1.50000000e-01, 2.14745884e-01, 7.10900474e-02,
       7.71208226e-02, 5.94059406e-02, 6.49350649e-02, 4.10958904e-01,
       1.62162162e-01, 9.78472619e-02, 1.69464555e-02, 2.82103311e-02,
       7.41162335e-02, 1.22448980e-01, 1.35135135e-01, 1.12190507e+01,
       2.14285714e-01, 1.25000000e-01, 5.00000000e-01, 2.40000000e-01,
      

In [ ]:
l, w, d = df['length'], df['width'], df['depth'] #TODO: убрать 

In [ ]:
cols_to_drop = ['length', 'width', 'depth']
df.drop(columns=cols_to_drop, inplace=True)

Вновь проверим корреляцию между получившимися дескриптоорами

In [ ]:
del df['Elongation']
del df['Flatness']

In [ ]:
del df['area']
del df['volume']

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 24 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Area/volume                                    567 non-null    float64
 1   Max/min(size)                                  568 non-null    float64
 2   org_coating_HAcceptors                         568 non-null    float64
 3   org_coating_LogP                               568 non-null    float64
 4   common_spin                                    568 non-null    float64
 5   Effective anisotropy constant (J/m3)           568 non-null    float64
 6   magnetic anisotropy constant (J/m3)            568 non-null    float64
 7   magnetocrystalline anisotropy constant (J/m3)  568 non-null    float64
 8   Anisotropy constant                            568 non-null    float64
 9   Crystalline                                    568 non

In [ ]:
df['Core_composition_formula'] = formula

In [ ]:
formula = df['Core_composition_formula'] #TODO: найти 

KeyError: 'Core_composition_formula'

In [ ]:
df.drop(columns=['Core_composition_formula', 'Stoichiometry', 'Elements'])

KeyError: "['Core_composition_formula', 'Stoichiometry', 'Elements'] not found in axis"

# Визуализации

In [ ]:
df['Core_composition_formula'] = formula

In [ ]:
df['Core_composition_formula'].unique()

array(['Au1Fe2O3', 'Co0.03Fe0.97Fe2O4', 'Co0.05Fe0.95Fe2O4',
       'Co0.1Fe0.9Fe2O4', 'Co0.24Mn0.15Fe0.61Fe2O4', 'Co0.3Fe0.7Fe2O4',
       'Co0.5Fe0.5Fe2O4', 'Co0.7Fe0.3Fe2O4', 'Co1Fe2O4', 'CoFe2O4',
       'Cu1Fe2O4', 'CuCuFe2O4', 'Fe0.5Mn0.5Fe2O4', 'Fe1Fe2O4',
       'Fe2.2C1Fe5C2', 'Fe2O3', 'Fe2O3Fe4(Fe(CN)6)3',
       'Fe2O3Fe4(Fe(CN)6)4', 'Fe3O4', 'Fe3O4CeO2', 'Fe3O4MnO2', 'Fe3O4Si',
       'FeV2O4', 'Mg0.1Mn0.9Fe2O4', 'Mg0.3Mn0.7Fe2O4', 'Mg0.4Mn0.6Fe2O4',
       'Mg0.5Mn0.5Fe2O4', 'MgFe2O4', 'Mn0.5Zn0.5Fe2O4', 'Mn1Fe2O4',
       'MnFe2O4', 'Ni0.5Zn0.5Fe2O4', 'Ni0.65Zn0.35Fe0.05Fe2O4',
       'Ni0.65Zn0.35Fe0.05Fe2O5', 'Ni0.65Zn0.35Fe0.05Fe2O6',
       'Ni0.65Zn0.35Fe0.05Fe2O7', 'Ni1Fe2O4', 'NiCo2O4Au0.44', 'NiFe2O4',
       'NiTe', 'PdFe2O3', 'Tb0.22Fe0.78Fe2O4', 'Tb0.5Fe0.5Fe2O4',
       'Tb0.75Fe0.25Fe2O4', 'Zn0.1Mn0.9Fe2O4', 'Zn0.2Mn0.8Fe2O4',
       'Zn0.3Co0.7Fe2O4', 'Zn0.3Mn0.7Fe2O4', 'Zn0.4Mn0.6Fe2O4',
       'Zn0.5Mn0.5Fe2O4', 'Zn0.6Mn0.4Fe2O4', 'Zn0.7Mn0.3Fe2O4',
      

Как видно многие формулы были записаны некоректно, поэтому исправим их немного

In [ ]:
df['link'] = link

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 26 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Area/volume                                        568 non-null    float64
 1   Max/min(size)                                      568 non-null    float64
 2   Core_composition_formula                           568 non-null    object 
 3   Stoichiometry                                      568 non-null    object 
 4   Elements                                           568 non-null    object 
 5   org_coating_HAcceptors                             568 non-null    float64
 6   org_coating_LogP                                   568 non-null    float64
 7   common_spin                                        568 non-null    float64
 8   Effective anisotropy constant (J/m3)               568 non-null    float64
 9   magnetic a

Для удобства переименуем столбец с 'magnetocrystalline anisotropy constant (J/m3)' на 'MAE'

In [ ]:
df = df.rename(columns = {'magnetocrystalline anisotropy energy (MAE) (J/m3)': 'MAE'})

Надо как-то это исправить

In [ ]:
formula = df['Core_composition_formula'] #TODO: удали

KeyError: 'Core_composition_formula'

In [ ]:
cols_to_drop = ['Core_composition_formula', 'Stoichiometry', 'Elements']
df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
df['Core_composition_formula'] = formula #TODO: аналогично
df['length'], df['width'], df['depth'] = l, w, d

In [ ]:
df['Area/volume'] = df['Area/volume'].fillna(df['Area/volume'].mean())

In [ ]:
nan_after = df.isna().sum()
print(f"Number of NaN values : {nan_after}")

Number of NaN values : Area/volume                                      0
Max/min(size)                                    0
org_coating_HAcceptors                           0
org_coating_LogP                                 0
common_spin                                      0
Effective anisotropy constant (J/m3)             0
magnetic anisotropy constant (J/m3)              0
magnetocrystalline anisotropy constant (J/m3)    0
Anisotropy constant                              0
Crystalline                                      0
MAE                                              0
Curie temperature (K)                            0
Spehricity                                       0
exchange constant (J/m) *10^-12                  0
Electronegativity core                           0
Space group                                      0
Demagnetization factors (Nd)                     0
Saturation magnetization (emu/g)                 0
Coercivity (Oe)                                  0
Remanenc

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 27 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Area/volume                                    568 non-null    float64
 1   Max/min(size)                                  568 non-null    float64
 2   org_coating_HAcceptors                         568 non-null    float64
 3   org_coating_LogP                               568 non-null    float64
 4   common_spin                                    568 non-null    float64
 5   Effective anisotropy constant (J/m3)           568 non-null    float64
 6   magnetic anisotropy constant (J/m3)            568 non-null    float64
 7   magnetocrystalline anisotropy constant (J/m3)  568 non-null    float64
 8   Anisotropy constant                            568 non-null    float64
 9   Crystalline                                    568 non

In [ ]:
df.to_csv('O_nan1.csv', index=False, float_format='%.3f')